<a href="https://colab.research.google.com/github/AnggaPradiktas/GrabAIChallenge/blob/master/train_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("drive/My Drive/Colab Notebooks")

In [0]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import keras
from training_keras import resnet152_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


Using TensorFlow backend.


In [0]:
img_width, img_height = 224,224
num_channels = 3
train_data = 'data/train'
valid_data = 'data/valid'
num_classes = 196
num_train_samples = 6549 #7839 
num_valid_samples = 1595 #2933
verbose = 1
batch_size = 16
num_epochs = 100000
patience = 50

In [0]:
if __name__ == '__main__':
    model = resnet152_model(img_height, img_width, num_channels, num_classes)
    
    train_data_gen = ImageDataGenerator(rotation_range=20.,
                                       width_shift_range=0.1,
                                       height_shift_range=0.1,
                                       zoom_range=0.2,
                                       horizontal_flip=True)
    valid_data_gen = ImageDataGenerator()
    
    tensor_board = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)
    log_file_path = 'logs/training.log'
    csv_logger = CSVLogger(log_file_path, append=False)
    early_stop = EarlyStopping('val_acc', patience=patience)
    reduce_lr = ReduceLROnPlateau('val_acc', factor=0.1, patience=int(patience/4), verbose=1)
    trained_model_path = 'model/model'
    model_names = trained_model_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
    model_checkpoint = ModelCheckpoint(model_names, monitor='val_acc', verbose=1, save_best_only=True)
    callbacks = [tensor_board, model_checkpoint, csv_logger, early_stop, reduce_lr]
    
    train_generator = train_data_gen.flow_from_directory(train_data, (img_width, img_height), batch_size=batch_size,
                                                        class_mode='categorical')
    valid_generator = valid_data_gen.flow_from_directory(valid_data, (img_width, img_height), batch_size=batch_size,
                                                        class_mode='categorical')
    
    model.fit_generator(train_generator, 
                        steps_per_epoch=num_train_samples/batch_size,
                        validation_data=valid_generator,
                        validation_steps=num_valid_samples/batch_size,
                        epochs=num_epochs,
                        callbacks=callbacks,
                        verbose=verbose)

Instructions for updating:
Colocations handled automatically by placer.
Found 7839 images belonging to 196 classes.
Found 2933 images belonging to 196 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100000
410/409 [==============================] - 2177s 5s/step - loss: 4.5095 - acc: 0.1063 - val_loss: 2.6312 - val_acc: 0.3694

Epoch 00001: val_acc improved from -inf to 0.36938, saving model to model/model.01-0.37.hdf5
Epoch 2/100000
410/409 [==============================] - 803s 2s/step - loss: 2.1216 - acc: 0.5067 - val_loss: 2.1908 - val_acc: 0.4418

Epoch 00002: val_acc improved from 0.36938 to 0.44179, saving model to model/model.02-0.44.hdf5
Epoch 3/100000
410/409 [==============================] - 275s 671ms/step - loss: 1.1034 - acc: 0.7242 - val_loss: 0.5620 - val_acc: 0.8462

Epoch 00003: val_acc improved from 0.44179 to 0.84625, saving model to model/model.03-0.85.hdf5
Epoch 4/100000
410/409 [==============================] - 275s 670ms/step - loss: 0.6671 